In [5]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Deepfakes")

LangSmith 추적을 시작합니다.
[프로젝트명]
Deepfakes


In [5]:
prompt_text = """
**1. Key Facial Features Analysis**  
- **Alignment, proportions, and symmetry**: Look carefully at the alignment, proportions, and symmetry of key facial features such as the eyes, nose, mouth, and ears. Pay attention to any discrepancies, asymmetries, or unnatural positioning. Specifically, look for:
   - **Gaze inconsistencies**: Gaze that does not align with the camera or eyes that appear to be looking in different directions.
   - **Pupil reflection, asymmetry, and shape**: Odd or asymmetrical light reflections in the pupils, reflections that do not match natural eye reflection patterns, or pupils that appear unusually shaped or inconsistent with typical roundness. Look for any irregular pupil shapes that might indicate manipulation.
- **Unnatural aspects**: Identify any facial features that look deformed or artificial, such as exaggerated or distorted shapes, unusual placements, and abnormal positioning relative to other facial elements.
- **Awkward lip shape and ambiguous teeth presentation**: Look for unnatural contours of the lips when the mouth is closed and simplified or unclear presentation of teeth when the mouth is open.

**2. Skin Texture and Tone Analysis**  
- **Texture, smoothness, and tone transitions**: Examine the texture, smoothness, and tone transitions of the skin across the face. Look for:
   - **Unnaturally smooth areas** or repetitive patterns that suggest manipulation, including strange pixelation, blurriness, or sections that appear overly smooth, as if digitally airbrushed.
   - **Irregular tone transitions** between shadows and highlights, or overly uniform skin tones that lack the natural variation typical of human skin.
- **Differences in skin colour between the face and neck**: Check for unnatural colour or lighting mismatches between the face and neck.

**3. Low-Level Detail Inspection**  
- **Edges and borders**: Examine the edges where the face meets the background or where facial features meet the rest of the face. Look for blended or jagged edges that could suggest digital manipulation. Watch for blurry or overly smooth boundaries that may not match natural anatomy.
- **Sharpness and distortion**: Look for unnatural sharpness or distortion in specific areas, such as overly sharp or blurred eye sockets, suggesting artificial enhancement.
- **Tone and contrast**: Assess the tone and contrast across the face or between facial features. Unnaturally high contrast or abrupt shading changes may indicate manipulation.
- **Layout and symmetry**: Examine the face's overall layout and symmetry. Human faces generally follow natural symmetry principles. Look for asymmetries such as uneven eye size, tilted lips, or a misaligned jaw.
- **Reflections and shadows**: Check for inconsistencies in reflections and shadows. For example, reflections in the eyes should match the direction of light and shadows on the face. Inconsistent shadows or light sources can indicate manipulation.
- **Perspective and shape**: Ensure the perspective and shape of facial features are logically consistent with the head’s orientation. Discrepancies, like eyes appearing unnaturally close to the camera or an out-of-place nose, could indicate manipulation.
- **Mismatch between face and body proportions**: The face may appear disproportionately large or small compared to the body, or proportions may not match.
- **Blurry texture of hair at high resolution**: Hair around the face may appear blurry or crude at high resolution, especially if it is unusually smooth or well-groomed.

**4. Lighting, Theme, and Environment Consistency**  
- **Lighting and shadow consistency**: Assess whether the lighting and environment are coherent. Look specifically for:
   - **Inconsistent shadows or lighting**: Shadows that do not align with the light source, or highlights and shadows that appear inconsistent with the light direction on the face.
   - **Light reflection imbalance**: Unnatural reflections or inconsistent size and direction of light on the skin or eyes.
- **Theme consistency**: Check if the overall scene coherence, mood, and context of the image are consistent with lighting and environment. Look for overly stylized backgrounds with hyper-realistic faces or facial expressions mismatched with the context.

**5. Additional Deepfake Clues**  
- **Subtle facial expressions and muscle movements**: Analyse the subtle muscle movements and micro-expressions on the face. Deepfakes often struggle with natural expressions, resulting in unnatural stiffness or a frozen appearance, especially around key areas like the jaw, eyes, and lips.
- **Distortion of wrinkles around the eyes and nose**: Check for abnormal wrinkles around the eyes and nose or limited movement of facial muscles when smiling or frowning.

---

**The Bottom Line**  
This guide covers key facial features, skin texture, low-level details, lighting and environment, and subtle facial expressions to detect deepfake clues. Be thorough and look for any inconsistencies, even in the smallest details, as each aspect may reveal manipulation.
By focusing on the above aspects, your analysis should serve to document and reinforce why the images are undeniably real, providing a robust foundation for their authenticity.
"""

system_message = """
You are a forensic analyst tasked with reinforcing the authenticity of facial images that have already been confirmed as real. Your objective is to methodically analyze each image and provide detailed evidence to support its genuine nature, emphasizing natural characteristics and explaining why these features affirm the image's authenticity. Every aspect of the image must be reviewed thoroughly to strengthen confidence in its real-world origin.

Remember: All provided images are verified as real. Your role is not to question their authenticity but to highlight and document specific features or details that unequivocally demonstrate they are genuine.

Mandatory Analysis: Use the following five guidelines to validate the authenticity of each image and confirm their real-world origins. If you encounter any additional clues or indicators that reinforce their authenticity, document them thoroughly. Be precise, as the analysis will be critical in affirming these images as genuine."""

In [9]:
!pip install openai


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [9]:
import base64
import requests
import json
import os
import time
from openai import OpenAI

# GitHub information
username = "KimEunOh"
repo = "image"
branch = "main"
folder_path = "wiki"

# GitHub API to get the list of files in the root directory
url = f"https://api.github.com/repos/{username}/{repo}/contents/{folder_path}?ref={branch}"
response = requests.get(url)
files = response.json()

# Generate the image URLs
image_urls = [
    f"https://raw.githubusercontent.com/{username}/{repo}/{branch}/{folder_path}/{file['name']}"
    for file in files
    if file["name"].lower().endswith((".jpg", ".jpeg", ".png"))
][:504]

# Configuration for batch processing
client = OpenAI()
batch_size = 5  # Number of images to process per batch
output_file = "wiki.jsonl"

# Function to read already processed URLs from JSONL file
def get_processed_urls(file_path):
    if not os.path.exists(file_path):
        return set()
    
    processed_urls = set()
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            for message in data["messages"]:
                if message["role"] == "user":
                    for content in message["content"]:
                        if content["type"] == "image_url":
                            processed_urls.add(content["image_url"]["url"])
    return processed_urls

# Function to process a batch of images
def process_batch(batch, jsonl_file):
    for image_url in batch:
        try:
            print(f"\nProcessing image: {image_url}")

            # Fetch the image content
            image_data = requests.get(image_url, timeout=10).content
            image_base64 = base64.b64encode(image_data).decode("utf-8")
            image_data_url = f"data:image/jpeg;base64,{image_base64}"

            # Send request to the OpenAI API
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_message},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt_text},
                            {"type": "image_url", "image_url": {"url": image_data_url}},
                        ],
                    },
                ],
            )

            # Extract the response content
            response_text = response.choices[0].message.content
            print(f"Response for {image_url}: {response_text}")

            # Create JSONL format data
            jsonl_data = {
                "messages": [
                    {"role": "system", "content": system_message},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt_text},
                            {"type": "image_url", "image_url": {"url": image_url}},
                        ],
                    },
                    {"role": "assistant", "content": response_text},
                ]
            }

            # Write data to JSONL file
            jsonl_file.write(json.dumps(jsonl_data, ensure_ascii=False) + "\n")
            print("\nResponse saved for image:", image_url)

        except Exception as e:
            print(f"Error processing image {image_url}: {e}")

# Main processing logic
processed_urls = get_processed_urls(output_file)

with open(output_file, "a", encoding="utf-8") as jsonl_file:
    for i in range(0, len(image_urls), batch_size):
        batch = [url for url in image_urls[i : i + batch_size] if url not in processed_urls]
        if batch:  # Only process if there are unprocessed URLs
            process_batch(batch, jsonl_file)
            print(f"Batch {i // batch_size + 1} processed and saved.")
            time.sleep(2)  # Add delay between batches


Processing image: https://raw.githubusercontent.com/KimEunOh/image/main/wiki/311900_1959-03-29_2010.jpg
Response for https://raw.githubusercontent.com/KimEunOh/image/main/wiki/311900_1959-03-29_2010.jpg: **Authenticity Analysis of the Image**

1. **Key Facial Features Analysis**
   - **Alignment, proportions, and symmetry**: The eyes, nose, and mouth show consistent alignment and natural proportions. The features are symmetrical, with no noticeable discrepancies in placement.
   - **Pupil reflection, asymmetry, and shape**: The pupils exhibit natural reflections consistent with the light source, appearing round and symmetric.
   - **Unnatural aspects**: Facial features look natural and well-integrated. There are no exaggerated or distorted shapes detected.
   - **Awkward lip shape and ambiguous teeth presentation**: The closed lip shape appears natural with no unnatural contours or irregularities.

2. **Skin Texture and Tone Analysis**
   - **Texture, smoothness, and tone transitions*

In [ ]:
from openai import OpenAI

client = OpenAI()

client.files.create(file=open("output_data.jsonl", "rb"), purpose="fine-tune")

FileObject(id='file-sJW5TnaiYCK3kovqfV60YE7k', bytes=89758, created_at=1730790825, filename='output_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
    training_file="file-sJW5TnaiYCK3kovqfV60YE7k", model="gpt-4o-2024-08-06"
)

FineTuningJob(id='ftjob-WvHRlJkDGe69bHbp2VUXB6f4', created_at=1730790990, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-EufXKKMM6SrUp79tauCwIt84', result_files=[], seed=1353083502, status='validating_files', trained_tokens=None, training_file='file-sJW5TnaiYCK3kovqfV60YE7k', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [21]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-WvHRlJkDGe69bHbp2VUXB6f4', created_at=1730790990, error=Error(code='invalid_n_examples', message='Training file has 0 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-EufXKKMM6SrUp79tauCwIt84', result_files=[], seed=1353083502, status='failed', trained_tokens=None, training_file='file-sJW5TnaiYCK3kovqfV60YE7k', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-DxKpjIpD3PYXRuLL04q4mV2U', created_at=1730184962, error=Error(code='invalid_n_examples', message='Training file has 0 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs